In [1]:
import matplotlib.pyplot as plt
import numpy as np
from mcparse import *

In [13]:
class Material:
    
    def __init__(self, name, mass_density, atomic_weight, mat_card_num, 
                 Cv=None, LiFrac=0, Li6_E=0, force_numden=False):
        
        self.name = name
        self.mass_density = mass_density
        self.atomic_weight = atomic_weight
        self.mnum = mat_card_num
        self.Cv = Cv
        self.fLi = LiFrac
        self.enrichment = Li6_E # Li-6 enrichment
        
        if force_numden == False:
            self.numden = self.mass_density * 6.022*10**23 / self.atomic_weight
        else:
            self.numden = force_numden
        
        master_material_cards = { "Li2TiO3" : f"m{mat_card_num}\n     "
                                            + f"3006.80c {round(LiFrac*Li6_E, 4)}\n     "     
                                            + f"3007.80c {round(LiFrac*(1-Li6_E), 4)}\n     "     
                                            +  "22048.80c 0.1666\n     "     
                                            +  "8016.80c 0.5000\n",
                                  "Li4SiO4" : f"m{mat_card_num}\n     "
                                            + f"3006.80c {round(LiFrac*Li6_E, 4)}\n     "     
                                            + f"3007.80c {round(LiFrac*(1-Li6_E), 4)}\n     "
                                            +  "8016.80c 0.4444\n     "
                                            +  "14028.80c 0.1111\n",
                                  "Li2ZrO3" : f"m{mat_card_num}\n     "
                                            + f"3006.80c {round(LiFrac*Li6_E, 4)}\n     "     
                                            + f"3007.80c {round(LiFrac*(1-Li6_E), 4)}\n     "
                                            +  "8016.80c 0.5\n     "
                                            +  "40091.80c 0.1666\n",
                                  "FLiBe"   : f"m{mat_card_num}\n     "
                                            + f"3006.80c {round(LiFrac*Li6_E, 4)}\n     "     
                                            + f"3007.80c {round(LiFrac*(1-Li6_E), 4)}\n     "
                                            +  "4009.80c 0.3333\n     "
                                            +  "9019.80c 0.3333\n     ",
                                  "W"       : f"m{mat_card_num}\n     "
                                            +  "74184.80c 1\n     ",
                                  "SiC"     : f"m{mat_card_num}\n     "
                                            +  "14028.80c 0.5\n     "
                                            +  "6012.80c 0.5\n     ",
                            "304L Steel"    : f"m{mat_card_num}\n     "
                                            +  "6000.80c 0.003635\n     "
                                            +  "25055.80c 0.019870\n     "
                                            +  "15031.80c 0.000793\n     "
                                            +  "16000.80c 0.000511\n     "
                                            +  "14000.80c 0.019434\n     "
                                            +  "24000.80c 0.199443\n     "
                                            +  "28000.80c 0.088343\n     "
                                            +  "26000.80c 0.667971\n     ",
                                  "BeO"     : f"m{mat_card_num}\n     "
                                            +  "4009.80c 0.5\n     "
                                            +  "8016.80c 0.498785\n     "
                                            +  "8017.80c 0.000190\n     "
                                            +  "8018.80c 0.001025\n     "}
        
        # Create material card
        if self.name in master_material_cards:
            self.mcard = master_material_cards[self.name]
        else:
            print(f"Unknown material: {self.name}. Create new material card.")
            
def sig(isotope, energy):
    
    '''Reads ENDF file for isotope
       and relevant cross section'''
    
    libraries = {"Li6"     : r"C:\Users\tanhe\Desktop\NSE 474\ENDF_Li6_n,alpha.txt",
                 "Isotope" : r"path\to\xsec\file"}
    
    if isotope in libraries:
        
        with open(libraries[isotope], "r") as f:
            data = f.readlines()[11:]
            
            # Separate Energy vs. Barns:
            energy_values = []
            barns  = []
            
            for line in data:
                energy_values.append(eval([x for x in data[data.index(line)].split(" ") if x][0]))
                barns.append(eval([x for x in data[data.index(line)].split(" ") if x][1]))
            
            # Calculate relevant cross section for our system
            # Assumes LINEAR interpolation!
            xsec = np.interp(energy, energy_values, barns)
            
        return xsec
    
    else:
        raise Exception("Invalid molecule.")
    
    return

def TCalc(N_iso, flux, sig, volume, endtime):
    
    '''Calculates total # of tritium
       atoms produced up to some time "t" '''
    
    t = np.linspace(0,endtime,10000)
    T3_tot = N_iso*(1 - np.exp(-flux*sig*volume*t))
    
    return T3_tot

# Material database

In [14]:
#                     NAME    MASS DENSITY,  ATOMIC WEIGHT   M CARD NUMBER    Li FRAC     Li6 ENRICHMENT
Li2TiO3 = Material("Li2TiO3",     3.43,         109.76,            1,      LiFrac=0.3334,  Li6_E=0.0485)
Li4SiO4 = Material("Li4SiO4",     2.221,        119.84,            2,      LiFrac=0.4444,  Li6_E=0.0485)
Li2ZrO3 = Material("Li2ZrO3",     4.21,         153.1,             3,      LiFrac=0.4,     Li6_E=0.0485)
SS_304L = Material("304L Steel",  8.00,        "ignore",           4,      force_numden=8.827*10**-2)
FLiBe   = Material("FLiBe",       1.91,          34.95,            5,      LiFrac=0.3334,  Li6_E=0.0485)
BeO     = Material("BeO",         3.01,          25.01,            6)

# Input deck optimization

In [52]:
# Things that still need to be added:
# - Case counter
# - New function to record what happens between each runs


def createInput(input_parameters=["list of initial properties"], caseNum=1, maxCases=100):
    
    ### Parameters ###
    HS_M = input_parameters[0]    # Heat shield material
    HS_T = input_parameters[1]    # Heat shield thickness
    B_M =  input_parameters[2]    # Blanket material
    B_T =  input_parameters[3]    # Blanket thicknes
    OS_M = input_parameters[4]    # Outer shield material
    OS_T = input_parameters[5]    # Outer shield thickness
    film_T = 1                    # Environment film thickness
    single_blanket_cell_T = input_parameters[6]
    core_rad = input_parameters[7]
    source_strength = input_parameters[8]
    nps = input_parameters[9]
    
    if B_T % single_blanket_cell_T == 0:
        num_blanket_cells = int(B_T / single_blanket_cell_T)
    else:
        raise Exception("Number of blanket cells must be integer (Blanket thickness / single blanket cell thickness)")
    
    # TERMINATE script if case number exceeds max
    if caseNum > maxCases:
        print("Case limit reached! Script terminated.")
        return
    
    filename = f"NSE474_HS_{HS_M.name}_B_{B_M.name}_OS_{OS_M.name}_case{caseNum}.i"
    with open(f'C:\\Users\\tanhe\\Desktop\\NSE 474\\inputs\\{filename}', 'w') as f:
        
        ### Label parameters in file ###
        f.write("Stellarator Optimization \n")
        f.write(f"c MATERIALS\n")
        f.write(f"c    Heat shield  - {HS_M.name} \n")
        f.write(f"c    Blanket      - {B_M.name} \n")
        f.write(f"c    Outer shield - {OS_M.name} \n")
        f.write(f"c LAYER THICKNESSES \n")
        f.write(f"c    Core Radius  - {core_rad} cm \n")
        f.write(f"c    Heat shield  - {HS_T} cm \n")
        f.write(f"c    Blanket      - {B_T} cm \n")
        f.write(f"c    Outer shield - {OS_T} cm \n")
        f.write(f"c    Environment  - {film_T} cm \n")
        f.write(f"c # OF BLANKET CELLS = {num_blanket_cells} \n")
        f.write(f"c BLANKET CELL THICKNESS = {single_blanket_cell_T} cm \n")
        f.write(f"c SOURCE STRENGTH = {source_strength}\n")
        f.write(f"c CASE NUMBER = {caseNum}\n")
        f.write(f"c =============================== \n")
        
        ### BEGIN CELLS ###
        
        # core
        f.write(f"1 0    -1 imp:n=1 $ core\n")
        
        # heat shield
        f.write(f"2 {HS_M.mnum} -{HS_M.mass_density} 1 -2 imp:n=1 $ HS\n")
        
        # blanket
        blanket_cell_list = [11]
        f.write(f"11 {B_M.mnum}  -{B_M.mass_density} 2 -11 imp:n=1 $ B1\n")  # First cell, verify surface!!
        for t in range(2,num_blanket_cells+1):
            cell = t + 10
            blanket_cell_list.append(cell)
            f.write(f"{cell} {B_M.mnum} -{B_M.mass_density} {cell-1} -{cell} imp:n=1 $ B{cell-10}\n")
            
        # outer shield
        f.write(f"{cell+1} {OS_M.mnum} -{OS_M.mass_density} {cell} -{cell+1} imp:n=1 $ OS\n")
        
        # film around outer shell to act as "environment"
        f.write(f"{cell+2} 0   {cell+1} -{cell+2} imp:n=1 $ film\n")
        
        # problem boundary
        f.write(f"{cell+3} 0   {cell+2}           imp:n=0 $ problem boundary\n")
        f.write("\n")
        
        ### BEGIN SURFACES ###
        
        # core
        f.write(f"1 so {core_rad}\n")
        
        # heat shield
        f.write(f"2 so {core_rad + HS_T}\n")
        
        # blanket
        for t in range(1,num_blanket_cells+1):
            cell = t + 10
            blanket_surface =  round((core_rad + HS_T) + (t * single_blanket_cell_T), 4)
            f.write(f"{cell} so {blanket_surface}\n")
            
        # outer shield
        f.write(f"{cell+1} so {blanket_surface + OS_T}\n")
        
        # film around outer shell to act as "environment"
        f.write(f"{cell+2} so {blanket_surface + OS_T + film_T}\n")       
        f.write("\n")
        
        ### BEGIN DATA CARDS ###
        # materials
        f.write(HS_M.mcard)
        f.write(B_M.mcard)
        f.write(OS_M.mcard)
        
        ## F4 tallies
        # f.write("f14:n 2\n") # Heat shield
        
        f.write("f24:n ")     # Blanket
        for i in range(num_blanket_cells):
            f.write(f"{blanket_cell_list[i]} ")
            if i % 6 == 0:
                f.write("\n     ")
        f.write("\n")
                
        f.write(f"f34:n {cell+2}\n") # Environment film
        
        ## F6 tallies
        f.write("f16:n 2\n") # Heat shield
        
        f.write("f26:n ")     # Blanket
        for i in range(num_blanket_cells):
            f.write(f"{blanket_cell_list[i]} ")
            if i % 6 == 0:
                f.write("\n     ")
        f.write("\n")
        
        ## SDEF card
        f.write("sdef pos=0 0 0 erg=2.5 par=1\n")
        
        ## NPS
        f.write(f"nps {nps}\n")
        
        
    return




def readOutP(output):
    
    # Grab details from beginning of input
    
    with open(output, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            if "c MATERIALS" in lines[i]:
                HS_M     = str(  [x for x in lines[i+1].split(" ") if x][5].replace("\n",""))
                B_M      = str(  [x for x in lines[i+2].split(" ") if x][4].replace("\n",""))
                OS_M     = str(  [x for x in lines[i+3].split(" ") if x][5].replace("\n",""))
                core_rad = float([x for x in lines[i+5].split(" ") if x][5].replace("\n",""))
                HS_T     = float([x for x in lines[i+6].split(" ") if x][5].replace("\n",""))
                B_T      = float([x for x in lines[i+7].split(" ") if x][4].replace("\n",""))
                OS_T     = float([x for x in lines[i+8].split(" ") if x][5].replace("\n",""))
                film_T   = float([x for x in lines[i+9].split(" ") if x][4].replace("\n",""))
                num_blanket_cells      = int(  [x for x in lines[i+10].split(" ") if x][7].replace("\n",""))
                blanket_cell_thickness = float([x for x in lines[i+11].split(" ") if x][6].replace("\n",""))
                source_strength        = float([x for x in lines[i+12].split(" ") if x][5].replace("\n",""))
                caseNumber             = int  ([x for x in lines[i+13].split(" ") if x][5].replace("\n",""))
            if "nps " in lines:
                nps = int(  [x for x in lines[i].split(" ") if x][1].replace("\n",""))
    
    # Initialize file for mcparse.py
    outp = ReadData(output)
    
    # Grab info
    BlanketCells      = outp.TallyCells(24)
    Blanket_F4_values = outp.TallyValues(tally_number=24)
    Blanket_F6_values = outp.TallyValues(tally_number=26)
    LastBlanketCell   = BlanketCells[-1]
    LastBlanket_F4_value = Blanket_F4_values[LastBlanketCell][0]
    LastBlanket_F4_error = Blanket_F4_values[LastBlanketCell][1]
    
    EnvironmentCells = outp.TallyCells(34)[0]
    Environment_F4   = outp.TallyValues(tally_number=34)
    Environment_F4_value = Environment_F4[EnvironmentCells][0]
    Environment_F4_error = Environment_F4[EnvironmentCells][1]
    
    # Record relevant data
    fluxBlanket = {}
    for cell in Blanket_F4_values:
        fluxBlanket.update( {cell : [Blanket_F4_values[cell][0] * source_strength,
                                     Blanket_F4_values[cell][1] ] } )
        
    fluxEnv = source_strength * Environment_F4_value
    
    ######### BEGIN PARAMETER CHANGES ##########
    
    # Thickness increments in cm
    HS_dx = 0
    B_dx  = 1
    OS_dx = 1
    
    # number of times to increase/decrease thickness
    HS_changes = 0
    B_changes  = 0
    OS_changes = 0
    
    # Determine bounds for certain criterion in cm
    maxHS_T = 10 
    minHS_T = 10 
    
    maxB_T  = 150 
    minB_T  = 1   
    
    maxOS_T = 100 
    minOS_T = 1
    
    maxFluxToEnv   = 0 # Check 10 CFR for limit
    minBlanketFlux = 1e-7 # No rhyme or reason, should it be zero instead?
    
    # Conditions
    maxBlanketReached = B_T > maxB_T
    minBlanketReached = B_T < minB_T
    
    maxOuterShieldReached = OS_T > maxOS_T
    minOuterShieldReached = OS_T < minOS_T
    
    ######### LOGIC CHECKS, GENERATE NEW PARAMETERS ###########
    
    # --- FLUX TO ENVIRONMENT TOO LARGE?
    if (Environment_F4_value > maxFluxToEnv):
        
        if not maxOuterShieldReached:
            print("Too much flux escaped. Increasing outer shield thickness...")
            OS_changes += 1
        elif not maxBlanketReached:
            print("Too much flux escaped. Outer shield limit reached. Increasing blanket thickness...")
            B_changes  += 1
            num_blanket_cells += 1
        else:
            print("Too much flux escaped, but both the blanket and outer\n"
                 +"shield have reached maximum thickness. No changes...")
        
        
    # --- FLUX TO ENVIRONMENT IS ACCEPTABLE, DECREASE OUTER SHIELD
    if (Environment_F4_value < maxFluxToEnv):
        
        if not minOuterShieldReached:
            print("No flux escaped. Decreasing outer shield thickness...")
            OS_changes -= 1
        else:
            print("No flux escaped and outer shield minimum thickness reached. No outer shield change.")
    
    # --- FLUX ESCAPING BLANKET
    if (LastBlanket_F4_value > minBlanketFlux):
        
        if not maxBlanketReached:
            print("Blanket not large enough, wasting flux. Increasing blanket thickness...")
            B_changes += 1
            num_blanket_cells += 1
        else:
            print("Blanket not large enough, but maximum thickness reached!\n")
            print("This is really bad! Either increase your max blanket limit\n")
            print("or decrease your assumed heat shield thickness!")
    
    # --- FLUX CANNOT BREACH ENTIRE BLANKET
    for cell in Blanket_F4_values:
        
        if Blanket_F4_values[cell][0] < minBlanketFlux:
            print("Not enough flux in blanket. Decreasing blanket thickness...")
            B_changes -= 1
            num_blanket_cells -= 1
            
            break
    
    # --- CALCULATE NEW THICKNESSES
    new_HS_T = HS_T + HS_changes*HS_dx
    new_B_T  = B_T  + B_changes *B_dx
    new_OS_T = OS_T + OS_changes*OS_dx
    
    new_parameter_list = [HS_M, new_HS_T, B_M, new_B_T, OS_M, new_OS_T,
                         blanket_cell_thickness, core_rad, source_strength, nps]
    
    ### CREATE NEW INPUT ###
    createInput(new_parameter_list, caseNum = caseNumber+1)



In [53]:
HS_M = BeO
B_M = Li2TiO3
OS_M = Li4SiO4
HS_T = 5
B_T = 50
OS_T = 10
single_blanket_cell_T = 1
core_rad = 50
source_strength = 1e9
nps = 100000

starting_parameters = [HS_M, HS_T, B_M, B_T, OS_M, OS_T, 
                       single_blanket_cell_T, core_rad, source_strength, nps]

createInput(input_parameters = starting_parameters)

In [54]:
output = r"C:\Users\tanhe\Desktop\NSE 474\inputs\outp"

readOutP(output)

IndexError: list index out of range